<a href="https://colab.research.google.com/github/RecoHut-Projects/recohut/blob/master/tutorials/modeling/T844978_ranking_models_ml1m_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CTR (Item ranking) models in Tensorflow on ML-1m

## **Step 1 - Setup the environment**

### **1.1 Install libraries**

In [ ]:
!pip install tensorflow==2.5.0

In [ ]:
!pip install -q -U git+https://github.com/RecoHut-Projects/recohut.git -b v0.0.5

### **1.2 Download datasets**

In [40]:
!pip install -q -U kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mrkmakr/criteo-dataset
!unzip criteo-dataset.zip

     |████████████████████████████████| 58 kB 4.3 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=1dbeac82a1c3f41b31d17564c04aaadbc847b1109c0ab974f22c2402a1b63de4
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
100% 4.31G/4.31G [01:22<00:00, 44.2MB/s]
100% 4.31G/4.31G [01:22<00:00, 56.3MB/s]
Archive:  criteo-dataset.zip
  inflating: dac/readme.txt          
  inflating: dac/test.txt            
  inflating: dac/train.txt           


### **1.3 Import libraries**

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

In [26]:
from recohut.transforms.datasets.criteo import create_criteo_dataset

from recohut.models.tf.fm import FM, FFM, NFM, AFM, DeepFM, xDeepFM
from recohut.models.tf.widedeep import WideDeep
from recohut.models.tf.deepcross import DeepCross
from recohut.models.tf.pnn import PNN
from recohut.models.tf.dcn import DCN

### **1.4 Set params**

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [37]:
class Args:
    def __init__(self, model='fm'):  
        self.file = 'dac/train.txt'
        self.read_part = True
        self.sample_num = 10000
        self.test_size = 0.2
        self.k = self.embed_dim = 8
        self.learning_rate = 0.001
        self.batch_size = 4096
        self.dnn_dropout = 0.5
        self.dropout = 0.5
        self.epochs = 2
        self.att_vector = 8
        self.mode = 'att'  # 'max', 'avg'
        self.activation = 'relu'
        self.embed_reg = 1e-5
        self.hidden_units = [256, 128, 64]
        self.cin_size = [128, 128]
        if model=='ffm':
            self.k = 10
            self.batch_size = 1024

## **Step 2 - Training & Evaluation**

### **2.1 FM**

In [6]:
args = Args(model='fm')

In [7]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                        read_part=args.read_part,
                                        sample_num=args.sample_num,
                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = FM(feature_columns=feature_columns, k=args.k)
    model.summary()
    # ============================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = '../save/fm_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ==============================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 39)]              0         
_________________________________________________________________
fm__layer (FM_Layer)         (None, 1)                 2183743   
_________________________________________________________________
tf.math.sigmoid (TFOpLambda) (None, 1)                 0         
Total params: 2,183,743
Trainable params: 2,183,743
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
18/18 [==============================] - 5s 180ms/step - loss: 0.5978 - auc: 0.5074 - val_loss: 0.5475 - val_auc: 0.5531
Epoch 2/2
5/5 [==============================] - 0s 19ms/step - loss: 0.5180 - auc: 0.6569
test AUC: 0.656945


### **2.2 FFM**

In [9]:
args = Args(model='ffm')

In [11]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                        read_part=args.read_part,
                                        sample_num=args.sample_num,
                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
model = FFM(feature_columns=feature_columns, k=args.k)
model.summary()
# ============================model checkpoint======================
# check_path = '../save/fm_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ============================Compile============================
model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                metrics=[AUC()])
# ==============================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 39)]              0         
_________________________________________________________________
ffm__layer (FFM_Layer)       (None, 1)                 17552382  
_________________________________________________________________
tf.math.sigmoid_1 (TFOpLambd (None, 1)                 0         
Total params: 17,552,382
Trainable params: 17,552,382
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
8/8 [==============================] - 103s 5s/step - loss: 2.3714 - auc_1: 0.5097 - val_loss: 1.6801 - val_auc_1: 0.5663
Epoch 2/2
2/2 [==============================] - 0s 98ms/step - loss: 1.8640 - auc_1: 0.6062
test AUC: 0.606180


### **2.3 Wide & Deep**

In [13]:
args = Args(model='widedeep')

In [15]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = WideDeep(feature_columns, hidden_units=args.hidden_units, dnn_dropout=args.dnn_dropout)
    model.summary()
    # ============================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = '../save/wide_deep_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ==============================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 39)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None,)              0           input_3[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None,)              0           input_3[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None,)              0           input_3[0][0]           

### **2.4 Deep Crossing**

In [16]:
args = Args(model='deepcross')

In [19]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = DeepCross(feature_columns, args.hidden_units)
    model.summary()
    # =========================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = 'save/deep_crossing_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ===========================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 39)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_39 (Sl (None,)              0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_40 (Sl (None,)              0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_41 (Sl (None,)              0           input_4[0][0]           

### **2.5 PNN**

In [20]:
args = Args(model='pnn')

In [22]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = PNN(feature_columns, args.hidden_units, args.dnn_dropout)
    model.summary()
    # =========================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = 'save/pnn_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ===========================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
The following Variables were used a Lambda layer's call (tf.tensordot), but
are not present in its tracked objects:
  <tf.Variable 'w_p:0' shape=(741, 8, 8, 256) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.tensordot_1), but
are not present in its tracked objects:
  <tf.Variable 'w_z:0' shape=(39, 8, 256) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.__operators__.add_3), but
are not present in its tracked objects:
  <tf.Variable 'l_b:0' sha

### **2.6 DCN**

In [23]:
args = Args(model='dcn')

In [25]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = DCN(feature_columns, args.hidden_units, dnn_dropout=args.dnn_dropout)
    model.summary()
    # =========================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = 'save/dcn_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ===========================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 39)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_117 (S (None,)              0           input_6[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_118 (S (None,)              0           input_6[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_119 (S (None,)              0           input_6[0][0]           

### **2.7 NFM**

In [27]:
args = Args(model='nfm')

In [28]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = NFM(feature_columns, args.hidden_units, dnn_dropout=args.dnn_dropout)
    model.summary()
    # =========================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = 'save/nfm_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ===========================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 39)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_156 (S (None,)              0           input_7[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_157 (S (None,)              0           input_7[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_158 (S (None,)              0           input_7[0][0]           

### **2.8 AFM**

In [34]:
args = Args(model='afm')

In [35]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = AFM(feature_columns, args.mode, args.att_vector, args.activation, args.dropout, args.embed_reg)
    model.summary()
    # =========================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = 'save/afm_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ===========================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)], # checkpoint,
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 39)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_195 (S (None,)              0           input_8[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_196 (S (None,)              0           input_8[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_197 (S (None,)              0           input_8[0][0]           

### **2.9 DeepFM**

In [ ]:
args = Args(model='deepfm')

In [36]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = DeepFM(feature_columns, hidden_units=args.hidden_units, dnn_dropout=args.dnn_dropout)
    model.summary()
    # ============================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = '../save/deepfm_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ==============================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint,
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 39)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_234 (S (None,)              0           input_9[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_235 (S (None,)              0           input_9[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_236 (S (None,)              0           input_9[0][0]           

### **2.10 xDeepFM**

In [38]:
args = Args(model='xdeepfm')

In [39]:
# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=args.file,
                                                        embed_dim=args.embed_dim,
                                                        read_part=args.read_part,
                                                        sample_num=args.sample_num,
                                                        test_size=args.test_size)
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = xDeepFM(feature_columns, args.hidden_units, args.cin_size)
    model.summary()
    # =========================Compile============================
    model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=args.learning_rate),
                    metrics=[AUC()])
# ============================model checkpoint======================
# check_path = 'save/xdeepfm_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# ===========================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=args.epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    batch_size=args.batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=args.batch_size)[1])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
The following Variables were used a Lambda layer's call (tf.__operators__.add_8), but
are not present in its tracked objects:
  <tf.Variable 'bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 39)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_273 (S (None,)              0           input_10[0][0]                   
_____________

## **Closure**

For more details, you can refer to https://github.com/RecoHut-Stanzas/S021355.

<a href="https://github.com/RecoHut-Stanzas/S021355/blob/main/reports/S021355.ipynb" alt="S021355_Report"> <img src="https://img.shields.io/static/v1?label=report&message=active&color=green" /></a> <a href="https://github.com/RecoHut-Stanzas/S021355" alt="S021355"> <img src="https://img.shields.io/static/v1?label=code&message=github&color=blue" /></a>

In [41]:
!pip install -q watermark
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-20 15:45:40

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

pandas    : 1.1.5
tensorflow: 2.5.0
IPython   : 5.5.0
numpy     : 1.19.5



---

**END**